# チェーンとLCEL

## 1. 準備

In [ ]:
# 必要なモジュールをインポート
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 環境変数の読み込み
load_dotenv()

# モデル名
MODEL_NAME = "gpt-5-mini"

## 2.基本のチェーン

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# プロンプトテンプレートの作成
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは{animal}らしく、語尾に{voice}などと付けて答えます。"),
    ("human", "{question}をする上でのポイントは？"),
])

# モデルの作成
model = ChatOpenAI(model=MODEL_NAME)

# 文字列で出力
str_output_parser = StrOutputParser()

# チェーンの作成。それぞれはRunnableである
chain = prompt | model | str_output_parser

# チェーンの実行
response = chain.invoke({"animal": "犬", "voice": "ワン！", "question": "英語学習"})

# 結果を表示
print(response)

## 3.任意の関数をチェーンにする

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain

# 与えられた文字列を逆順にして返す関数。@chainデコレータでRunnableLambdaになる
@chain
def reverse_string(message):
    return message[::-1]  # 文字列を反転

# プロンプトテンプレートの作成
prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}\n\n上記の質問について簡潔に回答してください。"),
])

# 文字列で出力
str_output_parser = StrOutputParser()

# チェーンの作成。@chainの代わりにここでRunnableLambda(reverse_string)としても良い。またはチェーンの一部なら自動変換される
my_chain = prompt | model | str_output_parser | reverse_string

# チェーンの実行
response = my_chain.invoke({"question": "こんにちは！"})

# 結果を表示
print(response)

## 4.複数のチェーンを並列につなげる

In [ ]:
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

# プロンプトテンプレートの作成
positive_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは楽観主義者です。ユーザーからの質問に対して常に前向きな回答をします。"),
    ("human", "{question}"),
])
negative_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは悲観主義者です。ユーザーからの質問に対して常に否定的な回答をします。"),
    ("human", "{question}"),
])

# モデルの作成
model = ChatOpenAI(model=MODEL_NAME)

# チェーンの作成
positive_chain = positive_prompt | model | str_output_parser
negative_chain = negative_prompt | model | str_output_parser
# 並行チェーンの作成
parallel_chain = RunnableParallel({
    "positive": positive_chain,
    "negative": negative_chain
})

# チェーンの実行
response = parallel_chain.invoke({"question": "AIの進化が人間に与える影響は？"})

# 結果を表示
print(json.dumps(response, ensure_ascii=False, indent=2))

In [ ]:
# 意見をまとめるプロンプトテンプレートの作成
opinion_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは平等主義者です。2つの意見を平等にまとめます。まとめた結果だけを出力します。"),
    ("human", "楽観的な意見: {positive}\n悲観的な意見: {negative}"),
])

opinion_chain = parallel_chain | opinion_prompt | model | str_output_parser

# チェーンの実行
response = opinion_chain.invoke({"question": "AIの進化が人間に与える影響は？"})

# 結果を表示
print(response)

## 5.入力をそのまま出力する

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough

# プロンプトテンプレートの作成
improve_prompt = ChatPromptTemplate.from_messages([
    ("system", "ユーザーからの質問を分析し、条件を定め、目的を明確にしたうえで、必要であればマークダウン記法を使い、タスクを細分化して最適な出力が得られるプロンプトを考えてください。最終的なプロンプトのみを出力してください。「ユーザーからの質問」に回答しないでください。"),
    ("human", "質問: {prompt}"),
])

# モデルの作成
model = ChatOpenAI(model=MODEL_NAME)

# プロンプト改善チェーンの作成
prompt_chain = improve_prompt | model | str_output_parser

# 並行チェーンの作成
parallel_chain = RunnableParallel({
    "original": RunnablePassthrough(),
    "improve": prompt_chain
})

# チェーンの実行
response = parallel_chain.invoke({"prompt": "AIの進化が人間に与える影響は？"})

# 結果を表示
print(json.dumps(response, ensure_ascii=False, indent=2))

## 6.ツールの使用

In [ ]:
from langchain_tavily import TavilySearch

# ツールの初期化
tavily_search = TavilySearch(
    max_results=2,                 # 取得する検索結果の数（k=5と同じ）
    search_depth="basic",          # "basic" (高速) か "advanced" (高品質)
    include_answer=False,          # Tavilyが生成した短い回答を含めない
    include_raw_content=False,     # HTMLの生コンテンツを含めるか（トークン消費増に注意）
    include_images=False,          # 画像URLを含めるか
    # include_domains=["go.jp"],   # 特定のドメインのみ検索する場合
    # exclude_domains=["wikipedia.org"] # 特定のドメインを除外する場合
)

# Tavilyの検索結果だけを、LLMが読みやすい形に整形する関数
def format_tavily_results(tavily_response: dict) -> str:
    results = tavily_response.get("results", [])
    if not results:
        return "（検索結果なし）"

    lines = []
    for i, r in enumerate(results, 1):
        title = r.get("title", "")
        content = r.get("content", "")
        url = r.get("url", "")
        lines.append(f"[{i}] {title}\n{content}\nsource: {url}")
    return "\n\n".join(lines)


In [ ]:
# tavily searchツールのテスト
response = tavily_search.invoke("奈良県のお土産と言えば？")
print(json.dumps(response, ensure_ascii=False, indent=2))

# セパレーター
print("-" * 20)

# 整形後の結果を確認
print(format_tavily_results(response))

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Tavily → results整形のチェーン
context_chain = tavily_search | format_tavily_results

prompt_text = """
あなたは以下の「知識」にのみ基づいて、ユーザーの質問に回答してください。
知識に書かれていない内容は推測せず、「知識からは分かりません」と答えてください。
可能であれば、回答の末尾に参照したsource番号（例：[1][2]）を付けてください。
最後にsource番号とurlを箇条書きで列挙してください。

# 知識:
{context}
"""

# プロンプトテンプレートの作成
prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_text),
    ("human", "質問: {question}"),
])

# モデルの作成
model = ChatOpenAI(model=MODEL_NAME)

chain = (
    {"context": context_chain,
     "question": RunnablePassthrough()}
    | prompt | model | str_output_parser
)

# チェーンの実行
response = chain.invoke("奈良県のお土産と言えば？")

# 結果を表示
print(response)

## 7.条件分岐

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.router import RouterRunnable
from pydantic import BaseModel, Field # 構造化データ定義用

# 判定ロジック用のクラス定義
class SearchDecision(BaseModel):
    """検索が必要かどうかを判定するためのモデル"""
    needs_search: bool = Field(description="Web検索が必要な場合はTrue、不要な場合はFalse")

In [ ]:
# 検索が必要かどうかを判断するチェーン
needs_search_prompt = ChatPromptTemplate.from_messages([
    ("user", "{question}\n\nこの質問に答えるためにWeb検索が必要ですか？")
])

needs_search_chain = (
    needs_search_prompt 
    | model.with_structured_output(SearchDecision) 
    | (lambda x: "search" if x.needs_search else "no_search") # 判定結果をルーティング用のkeyに変換
)

# 検索を実行するチェーン
context_chain = tavily_search | format_tavily_results

prompt_text = """
あなたは以下の「知識」にのみ基づいて、ユーザーの質問に回答してください。
知識に書かれていない内容は推測せず、「知識からは分かりません」と答えてください。
可能であれば、回答の末尾に参照したsource番号（例：[1][2]）を付けてください。
最後にsource番号とurlを箇条書きで列挙してください。

# 知識:
{context}
"""

search_answer_prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_text),
    ("human", "質問: {question}"),
])

search_chain = (
    {"context": context_chain, 
     "question": RunnablePassthrough()}
    | search_answer_prompt | model | str_output_parser
)

# 検索なしで回答するチェーン
no_search_answer_prompt = ChatPromptTemplate.from_messages([
    ("user", "{question}\n\n上記の質問に答えてください。")
])

no_search_chain = (
    no_search_answer_prompt 
    | model 
    | StrOutputParser()
)


In [ ]:
# RouterRunnable による分岐
# key に対応する Runnable を実行する
router = RouterRunnable(
    runnables={
        "search": search_chain,
        "no_search": no_search_chain,
    }
)

# RouterRunnable の入力は {"key": "...", "input": ...} の形
final_chain = (
    {"key": needs_search_chain, "input": RunnablePassthrough()}
    | router
)

In [ ]:
print("--- 実行: iphone16eの価格は？ ---")
# needs_search_chain が "search" を返す -> search_chain が実行される
print(final_chain.invoke("iphone16eの価格は？"))

print("\n--- 実行: 1+1は？ ---")
# needs_search_chain が "no_search" を返す -> no_search_chain が実行される
print(final_chain.invoke("1+1は？"))